# GeoNet FDSN webservice with Obspy - Event Service

**Refer to README.md in the folder containing this notebook for discussion of general information about GeoNet's FDSN webservers.**

In this tutorial we will look at geting seismic event data. The event service is only available for earthquake events.

By the end of this notebooks you should be able to find earthquakes events, and for a specific event find information like preferred location, types of magnitudes, and phase picks.

### Getting Started - Import Modules

In [1]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client as FDSN_Client
from obspy import read_inventory

### Define GeoNet FDSN client

In [2]:
client = FDSN_Client("GEONET")

## Accessing Earthquake Information
Use the **event** service to access earthquake parameters from the catalogue.

This example requests the Kaikoura earthquake and aftershocks for 24 hours following the event, within a 0.5 degree radius of the epicenter. It then prints a list of the events.

In [3]:
starttime = "2016-11-13 11:00:00.000"
endtime = "2016-11-14 11:00:00.000"
cat = client.get_events(starttime=starttime, endtime=endtime,latitude=-42.693,longitude=173.022,maxradius=0.5,minmagnitude=5)
print(cat)

10 Event(s) in Catalog:
2016-11-13T11:06:14.609261Z | -42.560, +173.094 | 5.63 M  | manual
2016-11-13T14:04:23.670948Z | -42.598, +173.016 | 5.14 M  | manual
2016-11-13T11:09:07.614197Z | -42.629, +172.950 | 5.30 M  | manual
2016-11-13T11:02:56.346094Z | -42.693, +173.022 | 7.82 Mw(mB) | manual
2016-11-13T12:25:55.365623Z | -42.670, +172.869 | 5.26 M  | manual
2016-11-13T12:23:09.937467Z | -42.452, +173.150 | 5.18 M  | manual
2016-11-13T11:08:14.955249Z | -42.503, +173.111 | 5.24 M  | manual
2016-11-13T22:19:32.198315Z | -42.352, +173.090 | 5.18 M  | manual
2016-11-14T00:34:23.359760Z | -42.465, +173.138 | 6.66 M  | manual
2016-11-14T05:16:32.460259Z | -42.390, +173.391 | 5.16 M  | manual


Single events can be requested using their PublicID, which is available from the GeoNet website. This example will demonstrate how to get additional information about the [Kaikoura Earthquake](http://www.geonet.org.nz/earthquake/2016p858000).

In [4]:
cat = client.get_events(eventid="2016p858000")
print(cat)

1 Event(s) in Catalog:
2016-11-13T11:02:56.346094Z | -42.693, +173.022 | 7.82 Mw(mB) | manual


In [5]:
ev = cat[0]
print(ev)

Event:	2016-11-13T11:02:56.346094Z | -42.693, +173.022 | 7.82 Mw(mB) | manual

	            resource_id: ResourceIdentifier(id="smi:nz.org.geonet/2016p858000")
	             event_type: 'earthquake'
	          creation_info: CreationInfo(agency_id='WEL(GNS_Primary)', author='scevent@akeqp01.geonet.org.nz', creation_time=UTCDateTime(2016, 11, 13, 11, 3, 27, 886738))
	    preferred_origin_id: ResourceIdentifier(id="smi:nz.org.geonet/Origin#20161116052725.210418.52361")
	 preferred_magnitude_id: ResourceIdentifier(id="smi:nz.org.geonet/Magnitude#20161116052734.469407.55057")
	                   ---------
	     event_descriptions: 1 Elements
	               comments: 1 Elements
	                  picks: 189 Elements
	             amplitudes: 200 Elements
	                origins: 1 Elements
	             magnitudes: 5 Elements
	     station_magnitudes: 200 Elements


Print out a summary of the information for the preferred origin.

In [6]:
origin = ev.origins[0]
print(origin)

Origin
	        resource_id: ResourceIdentifier(id="smi:nz.org.geonet/Origin#20161116052725.210418.52361")
	               time: UTCDateTime(2016, 11, 13, 11, 2, 56, 346094) [confidence_level=89.99999762, uncertainty=0.3114605068]
	          longitude: 173.0221405 [confidence_level=89.99999762, uncertainty=3.42338418]
	           latitude: -42.6925354 [confidence_level=89.99999762, uncertainty=2.342521173]
	              depth: 15114.45332 [confidence_level=89.99999762, uncertainty=3593.426501]
	          method_id: ResourceIdentifier(id="smi:nz.org.geonet/LOCSAT")
	     earth_model_id: ResourceIdentifier(id="smi:nz.org.geonet/iasp91")
	            quality: OriginQuality(associated_phase_count=189, used_phase_count=94, associated_station_count=186, used_station_count=93, depth_phase_count=0, standard_error=1.822964596, azimuthal_gap=54.98503113, minimum_distance=0.2205573916, maximum_distance=7.640860558, median_distance=5.064006805)
	 origin_uncertainty: OriginUncertainty(horizontal_u

List all available magnitudes and their associated uncertainties. This event had a magnitude for five magnitude types.

In [7]:
for m in range(len(ev.magnitudes)):
    if 'uncertainty' in ev.magnitudes[m].mag_errors and ev.magnitudes[m].mag_errors['uncertainty'] != None and ev.magnitudes[m].resource_id == ev.preferred_magnitude_id:
        print('%s = %f +/- %f - Preferred magnitude' % (ev.magnitudes[m].magnitude_type, ev.magnitudes[m].mag, ev.magnitudes[m].mag_errors['uncertainty']))
    elif 'uncertainty' in ev.magnitudes[m].mag_errors and ev.magnitudes[m].mag_errors['uncertainty'] != None:
        print('%s = %f +/- %f' % (ev.magnitudes[m].magnitude_type, ev.magnitudes[m].mag, ev.magnitudes[m].mag_errors['uncertainty']))
    else:
        print('%s = %f' % (ev.magnitudes[m].magnitude_type, ev.magnitudes[m].mag))

MLv = 7.363463 +/- 0.249774
ML = 7.763918 +/- 0.342303
mB = 7.692600 +/- 0.190699
Mw(mB) = 7.820380 +/- 0.400000 - Preferred magnitude
M = 7.762542


List all arrivals used to locate the earthquake.

In [8]:
print(origin.arrivals[0])
print(ev.picks[0])

Arrival
	   resource_id: ResourceIdentifier(id="smi:nz.org.geonet/20161113.110307.63-AIC-NZ.LTZ.10.HHZ#Origin#20161116052725.210418.52361")
	       pick_id: ResourceIdentifier(id="smi:nz.org.geonet/20161113.110307.63-AIC-NZ.LTZ.10.HHZ")
	         phase: 'P'
	       azimuth: 260.5674491
	      distance: 0.5588204424
	 time_residual: 0.253331
	   time_weight: 1.0
Pick
	     resource_id: ResourceIdentifier(id="smi:nz.org.geonet/20161113.110307.63-AIC-NZ.LTZ.10.HHZ")
	            time: UTCDateTime(2016, 11, 13, 11, 3, 7, 638393)
	     waveform_id: WaveformStreamID(network_code='NZ', station_code='LTZ', channel_code='HHZ', location_code='10')
	       filter_id: ResourceIdentifier(id="smi:nz.org.geonet/BW(4,2.5,15)")
	       method_id: ResourceIdentifier(id="smi:nz.org.geonet/AIC")
	      phase_hint: 'P'
	 evaluation_mode: 'automatic'
	   creation_info: CreationInfo(agency_id='WEL(GNS_Primary)', author='scautopick@akeqp01.geonet.org.nz', creation_time=UTCDateTime(2016, 11, 13, 11, 3, 10, 430

In [9]:
for p in range(len(ev.picks)):
    for a in range(len(origin.arrivals)):
        if ev.picks[p].resource_id == origin.arrivals[a].pick_id:
            if origin.arrivals[a].time_weight > 0:
                print(ev.picks[p].time, ev.picks[p].waveform_id['station_code'], origin.arrivals[a].distance, origin.arrivals[a].phase, origin.arrivals[a].time_residual)

2016-11-13T11:03:07.638393Z LTZ 0.5588204424 P 0.253331
2016-11-13T11:03:09.408392Z AMCZ 0.5548961874 P 2.125621
2016-11-13T11:03:13.023198Z THZ 0.9340822023 P -1.184325
2016-11-13T11:03:14.813361Z OXZ 0.9584892308 P 0.159378
2016-11-13T11:03:15.678398Z MQZ 1.048513991 P -0.480021
2016-11-13T11:03:16.208382Z OKCZ 1.031074393 P 0.351306
2016-11-13T11:03:17.693129Z INZ 1.159989466 P -0.353515
2016-11-13T11:03:18.048388Z BSWZ 1.164270388 P -0.017894
2016-11-13T11:04:02.968388Z ARHZ 4.555322735 P -1.750663
2016-11-13T11:04:03.353127Z NMHZ 4.593566012 P -1.880129
2016-11-13T11:04:04.038394Z WATZ 4.48255132 P 0.374367
2016-11-13T11:04:04.048392Z MRHZ 4.636222848 P -1.75075
2016-11-13T11:04:06.433130Z WHZ 4.839242842 P -2.237686
2016-11-13T11:04:06.538392Z SYZ 4.733693963 P -0.632041
2016-11-13T11:04:06.608390Z MTHZ 4.806288451 P -1.541351
2016-11-13T11:04:06.678390Z RAHZ 4.869376797 P -2.348056
2016-11-13T11:04:06.698402Z WHHZ 4.816871884 P -1.616553
2016-11-13T11:04:07.038388Z ALRZ 4.837754